In [26]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [27]:
file_in = "../Data_Sourcing/nfl_with_college_stat_links.csv"
df = pd.read_csv(file_in)
print(f'loaded csv: {file_in}')

loaded csv: ../Data_Sourcing/nfl_with_college_stat_links.csv


In [28]:
display(df.iloc[:, :10].head())
display(df.iloc[:, 10:].head())
# display(df.info())
display(df.columns)
display(len(df))

,player,hof_yr,all_star,SBs,wAV,AV,record,games,win_%,pass_yards
0,Steve Bartkowski,NaN,['2x Pro Bowl'],0.0,100.0,84,59-68-0,129.0,46.46,24124.0
1,Steve Beuerlein,NaN,['1x Pro Bowl'],1.0,100.0,85,47-55-0,147.0,46.08,24046.0
2,Archie Manning,NaN,['2x Pro Bowl'],0.0,100.0,94,35-101-3,151.0,25.74,23911.0
3,Andrew Luck,NaN,['4x Pro Bowl'],0.0,100.0,80,53-33-0,86.0,61.63,23671.0
4,Norm Van Brocklin,1960.0,"['9x Pro Bowl', '1x All-Pro']",0.0,100.0,17,61-36-4,140.0,62.89,23611.0


,TD,comp_%,YPA,int,draft_yr,draft_team,rd,pick,teams,high_school,college,college_stats,ht,wt,hand,wins,loss
0,156.0,55.9,7.0,144.0,1975.0,Atlanta Falcons,1.0,1.0,"['Atlanta Falcons 1975-1985', 'Los Angeles Ram...",Buchser,California,https://www.sports-reference.com/cfb/players/s...,6-4,216.0,Right,59.0,68.0
1,147.0,56.9,7.2,112.0,1987.0,Los Angeles Raiders,4.0,110.0,"['Los Angeles Raiders 1988-1989', 'Dallas Cowb...",Servite,Notre Dame,https://www.sports-reference.com/cfb/players/s...,6-3,220.0,Right,47.0,55.0
2,125.0,55.2,6.6,173.0,1971.0,New Orleans Saints,1.0,2.0,"['New Orleans Saints 1971-1982', 'Houston Oile...",Drew,Mississippi,https://www.sports-reference.com/cfb/players/a...,6-3,212.0,Right,35.0,101.0
3,171.0,60.8,7.2,83.0,2012.0,Indianapolis Colts,1.0,1.0,['Indianapolis Colts 2012-2018'],Stratford,Stanford,https://www.sports-reference.com/cfb/players/a...,6-4,240.0,Right,53.0,33.0
4,173.0,53.6,8.2,178.0,1949.0,Los Angeles Rams,4.0,37.0,"['Los Angeles Rams 1949', 'Los Angeles Rams 19...",Acalanes,Oregon,https://www.sports-reference.com/cfb/players/n...,6-1,190.0,Right,61.0,36.0


Index(['player', 'hof_yr', 'all_star', 'SBs', 'wAV', 'AV', 'record', 'games',
       'win_%', 'pass_yards', 'TD', 'comp_%', 'YPA', 'int', 'draft_yr',
       'draft_team', 'rd', 'pick', 'teams', 'high_school', 'college',
       'college_stats', 'ht', 'wt', 'hand', 'wins', 'loss'],
      dtype='object')

472

In [29]:
nfl_df = df[['player','college_stats']]
nfl_df.head()

,player,college_stats
0,Steve Bartkowski,https://www.sports-reference.com/cfb/players/s...
1,Steve Beuerlein,https://www.sports-reference.com/cfb/players/s...
2,Archie Manning,https://www.sports-reference.com/cfb/players/a...
3,Andrew Luck,https://www.sports-reference.com/cfb/players/a...
4,Norm Van Brocklin,https://www.sports-reference.com/cfb/players/n...


In [30]:
players = df['player'].tolist()
links = df['college_stats'].tolist()
player_link_dict = dict(zip(players, links))

print(player_link_dict)


{'Steve Bartkowski': 'https://www.sports-reference.com/cfb/players/steve-bartkowski-1.html', 'Steve Beuerlein': 'https://www.sports-reference.com/cfb/players/steve-beuerlein-1.html', 'Archie Manning': 'https://www.sports-reference.com/cfb/players/archie-manning-1.html', 'Andrew Luck': 'https://www.sports-reference.com/cfb/players/andrew-luck-1.html', 'Norm Van Brocklin': 'https://www.sports-reference.com/cfb/players/norm-van-brocklin-1.html', 'Otto Graham': 'https://www.sports-reference.com/cfb/players/otto-graham-1.html', 'Lynn Dickey': 'https://www.sports-reference.com/cfb/players/lynn-dickey-1.html', 'Bernie Kosar': 'https://www.sports-reference.com/cfb/players/bernie-kosar-1.html', 'Marc Bulger': 'https://www.sports-reference.com/cfb/players/marc-bulger-1.html', 'Roger Staubach': 'https://www.sports-reference.com/cfb/players/roger-staubach-1.html', 'Babe Parilli': 'https://www.sports-reference.com/cfb/players/babe-parilli-1.html', 'Michael Vick': 'https://www.sports-reference.com/c

In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def get_soup(url):
    response = requests.get(url)
    if response.status_code == 200:
        return BeautifulSoup(response.content, 'html.parser')
    else:
        print(f"Failed to retrieve webpage. Status code: {response.status_code}")
        return None

def parse_career_row(soup, table_id):
    table = soup.find('table', id=table_id)
    career_footer = table.find('tfoot')
    career_row = career_footer.find('tr')
    headers = [th.text for th in table.find('thead').find_all('th')]
    career_data = [td.text.strip() for td in career_row.find_all(['th', 'td'])]
    return headers, career_data

def get_awards(soup, table_id):
    table = soup.find('table', id=table_id)
    awards = []
    for row in table.find_all('tr'):
        award_cell = row.find('td', {'data-stat': 'awards'})
        if award_cell:
            awards.extend([a.text for a in award_cell.find_all('a')])
    return ', '.join(awards)

def career_row_to_dataframe(headers, career_data, awards):
    df = pd.DataFrame([career_data], columns=headers)
    df['awards'] = awards
    return df

# Sample dictionary of players and their college stats URLs


# Dictionary to store DataFrames
player_dataframes = {}

# Loop through each player and their college_stats URL
for idx, (player, link) in enumerate(player_link_dict.items()):
    if idx > 5:
        break
    print(f"Player: {player}, College Stats URL: {link}")
    soup = get_soup(link)
    if soup:
        headers, career_data = parse_career_row(soup, 'passing_standard')
        awards = get_awards(soup, 'passing_standard')
        df = career_row_to_dataframe(headers, career_data, awards)
        player_dataframes[player] = df
        print(f"Career data and awards for {player} successfully retrieved and stored.")
    else:
        print(f'No soup for you, {player} at {link}')
    time.sleep(8)  # To avoid overwhelming the server with requests

# Now player_dataframes contains DataFrames for each player's career stats and awards
print(player_dataframes['Steve Bartkowski'])


Player: Steve Bartkowski, College Stats URL: https://www.sports-reference.com/cfb/players/steve-bartkowski-1.html


ValueError: 20 columns passed, passed data had 17 columns

In [20]:
player_dataframes['Steve Bartkowski']

,Season,Team,Conf,Class,Pos,G,Cmp,Att,Cmp%,Yds,TD,TD%,Int,Int%,Y/A,AY/A,Y/C,Y/G,Rate,Awards
0,1972,California,Pac-8,,QB,11,70,165,42.4,944,4,2.4,13,7.9,5.7,2.66,13.5,85.8,82.7,
1,1973,California,Pac-8,,QB,11,61,129,47.3,910,4,3.1,7,5.4,7.1,5.23,14.9,82.7,105.9,
2,1974,California,Pac-8,,QB,11,182,325,56.0,2580,12,3.7,7,2.2,7.9,7.71,14.2,234.5,130.6,"H-10,AA"


In [6]:
import requests
from bs4 import BeautifulSoup
import time

def get_soup(url):
    response = requests.get(url)
    if response.status_code == 200:
        return BeautifulSoup(response.content, 'html.parser')
    else:
        print(f"Failed to retrieve webpage. Status code: {response.status_code}")
        return None
    
# ----------------------------------------------------------------------------------

for idx, (player, link) in enumerate(player_link_dict.items()):
    if idx > 2:
        break
    print(f"Player: {player}, College Stats URL: {link}")
    time.sleep(8)
    soup = get_soup(link)
    if soup:
        pass 
    else: 
        print(f'no soup for you, {player} at {link}')  





Player: Steve Bartkowski, College Stats URL: https://www.sports-reference.com/cfb/players/steve-bartkowski-1.html
Player: Steve Beuerlein, College Stats URL: https://www.sports-reference.com/cfb/players/steve-beuerlein-1.html
Player: Archie Manning, College Stats URL: https://www.sports-reference.com/cfb/players/archie-manning-1.html
Player: Andrew Luck, College Stats URL: https://www.sports-reference.com/cfb/players/andrew-luck-1.html
Player: Norm Van Brocklin, College Stats URL: https://www.sports-reference.com/cfb/players/norm-van-brocklin-1.html
Player: Otto Graham, College Stats URL: https://www.sports-reference.com/cfb/players/otto-graham-1.html
